# SMT

## Import and utils

In [1]:
from z3 import *

## Read values from the file

In [2]:
# Read the file and initialize the variables
with open("../data/inst03.dat", "r") as file:
    lines = file.readlines()

In [3]:
# CREATE SOLVER INSTANCE
#solver = Solver()
solver = Optimize()

## Decision variables

In [4]:
# Parse the values from the file
m = int(lines[0].strip())  # Number of couriers
n = int(lines[1].strip())  # Number of items
l_values = list(map(int, lines[2].split()))  # Maximum load for each courier
l = Array('l', IntSort(), IntSort())
for i in range(m):
    solver.add(l[i] == l_values[i])

s_values = list(map(int, lines[3].split()))  # Size of each item
s = Array('s', IntSort(), IntSort())

for i in range(n):
    solver.add(s[i] == s_values[i])

D_values = [list(map(int, line.split())) for line in lines[4:]]  # Distance matrix
D = Array('D', IntSort(), ArraySort(IntSort(), IntSort()))  # Routes for each courier

# Set the matrix of distances D to the default values
for i in range(n+1):
#    #a_i = Array('a_{i}', IntSort(), IntSort())
    for j in range(n+1):
        solver.add(D[i][j] == D_values[i][j])

routes = Array('routes', IntSort(), ArraySort(IntSort(), IntSort()))  # Routes for each courier

#D = [[Int(f"D_{i}_{j}") for j in range(n + 1)] for i in range(n + 1)]
#for i in range(n + 1):
#    for j in range(n + 1):
#        D[i][j] = D_values[i][j]
#print(s)

In [5]:
#Useful functions
def z3_max(vector):
    maximum = vector[0]
    for value in vector[1:]:
        maximum = If(value > maximum, value, maximum)
    return maximum

def z3_min(vector):
    minimum = vector[0]
    for i in range(1,n):
        value = vector[i]
        minimum = If(value < minimum, value, minimum)
    return minimum

In [6]:
# DOMAIN CONSTRAINTS
for i in range(m):
    for t in range(n):
        solver.add(And(routes[i][t] >= 1, routes[i][t] <= n + 1))

## Constraints

In [7]:
# TODO: FIX CONSTRAINTS
# All the values from 1 to n need to appear exactly once in the routes matrix
for p in range(1,n+1):
   solver.add(Sum([If(routes[i][j] == p, 1, 0) for i in range(m) for j in range(n+1)]) == 1)


for i in range(m):
    
    #The courier must finish at the origin point
    solver.add(routes[i][n] == n+1)
    
    #Constraint to force that the total size of items assigned to each courier cannot exceed their maximum load size and constraint to force that the total size of items assigned to each courier is at least the load of the min value of item sizes
    sums = Sum([If(routes[i][j] != n+1, s[routes[i][j] -1], 0) for j in range(n+1)])
    solver.add(And(sums <= l[i], sums>=z3_min(s)))
    
    
    #Simmetry breaking
    #Constraint to force the first value of each row of the matrix to be different from n+1
    solver.add(routes[i][0] != n+1)

    #Constraint to force all the numbers after the first n+1 to be also n+1
    for j in range(n):
        solver.add(If(routes[i][j] == n+1, routes[i][j+1] == n+1, True))
    
     
##IMPLICIT CONSTRAINT
## Constraint to force exactly (m * (n + 1)) - n n+1s
solver.add(Sum([If(routes[i][j] == n + 1, 1, 0) for i in range(m) for j in range(n+1)]) == (m * (n + 1)) - n)

In [8]:
# OBJECTIVE FUNCTION
# Application of the same objective function above to all couriers
#dist_courier = [(Sum( [D[routes[i][j]][routes[i][j + 1]] for j in range(n)] )+ D[n + 1][routes[i][0]] )  for i in range(m)]
dist_courier = [(Sum( [D[routes[i][j]-1][routes[i][j + 1]-1] for j in range(n)] )+ D[n][routes[i][0]-1] )  for i in range(m)]


maximum = z3_max(dist_courier)



In [9]:
solver.minimize(maximum)
if solver.check() == sat:
    model = solver.model()
    length = model.evaluate(maximum)
    
    
    routes_sol = []
    for i in range(m):
        routes_sol.append([model.evaluate(routes[i][j]).as_long() for j in range(n+1)])
    print(routes_sol)
    print(length)
else:
    print("no sat")
#TODO: Create function that prints the results


[[3, 6, 5, 8, 8, 8, 8, 8], [4, 2, 8, 8, 8, 8, 8, 8], [1, 7, 8, 8, 8, 8, 8, 8]]
12


In [10]:
print('---n---')
print(n)
print("---SIZES---")
print([model.evaluate(s[i]) for i in range(n) ])
print("---LOADS---")
print([model.evaluate(l[i]) for i in range(m) ])
print('---DISTANCES---')  
for i in range(n+1):
    row = []
    for j in range(n+1):
        a = model.evaluate(D[i][j]).as_long()
        row.append(a)
    print(row)

print('---ROUTES---')  
for i in range(m):
    row = []
    for j in range(n+1):
        a = model.evaluate(routes[i][j]).as_long()
        row.append(a)
    print(row)

print('---DIST COURIER---')  
print([model.evaluate(dist_courier[i]) for i in range(m)])




---n---
7
---SIZES---
[3, 2, 6, 8, 5, 4, 4]
---LOADS---
[15, 10, 7]
---DISTANCES---
[0, 3, 3, 6, 5, 6, 6, 2]
[3, 0, 6, 3, 4, 7, 7, 3]
[3, 4, 0, 7, 6, 3, 5, 3]
[6, 3, 7, 0, 5, 6, 7, 4]
[5, 4, 6, 3, 0, 3, 3, 3]
[6, 7, 3, 6, 3, 0, 2, 4]
[6, 7, 5, 6, 3, 2, 0, 4]
[2, 3, 3, 4, 3, 4, 4, 0]
---ROUTES---
[3, 6, 5, 8, 8, 8, 8, 8]
[4, 2, 8, 8, 8, 8, 8, 8]
[1, 7, 8, 8, 8, 8, 8, 8]
---DIST COURIER---
[12, 10, 12]
